In [1]:
import os
from PIL import Image
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import transforms
import matplotlib.pyplot as plt

import preprocess as p
import display as  disp



In [4]:
!pip install matplotlib
#python3 -m pip install --upgrade pip  en la terminal

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
!pip install seaborn

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
#Prueba de visualización de gráficas
epochs = [1, 2, 3]
train_loss_values = [1, 5, 3]
train_acc_values= [1.5, 2.5, 3.5]
val_loss_values = [1.5, 2.5, 3.5]
val_acc_values = [1, 2, 3]
# Crear la figura y los ejes
plt.figure(figsize=(12, 6))
# Crear la figura y los ejes
plt.figure(figsize=(12, 6))

    # Gráfica del Loss
plt.subplot(1, 2, 1) 
plt.plot(epochs, train_loss_values, color='red', label='Train Loss')
plt.plot(epochs, val_loss_values, color='orange', label='Validation Loss')
    # Encontrar el mínimo para train y validation
train_min_idx = train_loss_values.index(min(train_loss_values))
val_min_idx = val_loss_values.index(min(val_loss_values))
    #Marcar los mínimos
plt.plot(epochs[train_min_idx], train_loss_values[train_min_idx], 'ro', label=f'Train Min: {train_loss_values[train_min_idx]:.2f}')
plt.plot(epochs[val_min_idx], val_loss_values[val_min_idx], 'o', color='orange', label=f'Validation Min: {val_loss_values[val_min_idx]:.2f}')
plt.title('Loss Graphic')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()

    # Gráfica del Accuracy
plt.subplot(1, 2, 2)  
plt.plot(epochs, train_acc_values, color='blue', label='Train Accuracy')
plt.plot(epochs, val_acc_values, color='green', label='Validation Accuracy')
    # Encontrar el máximo para train y validation
train_max_idx = train_acc_values.index(max(train_acc_values))
val_max_idx = val_acc_values.index(max(val_acc_values))
    #Marcar los mínimos
plt.plot(epochs[train_max_idx], train_acc_values[train_max_idx], 'bo', label=f'Train Max: {train_acc_values[train_max_idx]:.2f}')
plt.plot(epochs[val_max_idx], val_acc_values[val_max_idx], 'go', label=f'Validation Max: {val_acc_values[val_max_idx]:.2f}')
plt.title('Accuracy Graphic')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.grid(True)
plt.legend()

In [2]:
arch = "3C"
lr = 0.001
da = "NO"

name = f"{arch}_{lr}_{da}"
print(f"{name}")

3C_0.001_NO


In [3]:
name = disp.crear_nombre_modelo(arch, lr, da)
print(f"{name}")

1_3C_2_NO


In [6]:
save = False
print(f"{save}")

False


In [7]:
print("[VALIDACIÓN]")
if save == True:
    print("Epoch [{epoch+1}/{num_epochs}] - Loss: {val_epoch_loss:.4f}, Accuracy: {val_epoch_accuracy:.2f}% - Best")
else:
    print("Epoch [{epoch+1}/{num_epochs}] - Loss: {val_epoch_loss:.4f}, Accuracy: {val_epoch_accuracy:.2f}%")
        
save = False 

[VALIDACIÓN]
Epoch [{epoch+1}/{num_epochs}] - Loss: {val_epoch_loss:.4f}, Accuracy: {val_epoch_accuracy:.2f}%
